# Analysing experimental data

We need:

- **Data** need to be read and prepared.
 - Data may be a single dataset (usually in a *dataArray*) or several of these (in a *dataList*) like
   multiple measurements with same or changing parameters (e.g. wavevectors).
   Coordinates are in *.X* and values in *.Y*
 - 2D data (e.g. a detector image with 2 dimensions) need to be transformed to coordinates
   *.X*,*.Z* with values in *.Y*. This also gives pixels coordinates in an image a physical
   interpretation as e.g. wavevectors.
   See examples ['2D fitting'](https://jscatter.readthedocs.io/en/latest/examples.html#d-fitting) and [`Fitting the 2D scattering of a lattice`](https://jscatter.readthedocs.io/en/latest/examples.html#fitting-the-2d-scattering-of-a-lattice)
 - Attributes need to be extracted from read data (from Comments or filename or from a list..)
   In the below example the Temperature is stored in the data as attribute.

- A **model**, which can be defined in different ways.

 - See below or in the notebook `Jscatter_BuildModels` for different ways.

  - Please avoid using lists as parameters as list are used to discriminate
    between common parameters (a single float) and individual fit parameters
    (a list of float for each) in dataList.

- **Fit algorithm**

  We use methods from the scipy.optimize module that are incorporated in the *.fit*
  method of *dataArray/dataList*. *.fit* supports different fit algorithms
  (see dataList.fit Examples how to choose and about speed differences):
 - method='leastsq' (default) is a wrapper around MINPACK’s lmdif and lmder, which is a modification
   of the [Levenberg-Marquardt](https://en.wikipedia.org/wiki/Levenberg-Marquardt_algorithm) algorithm.
   This is what you usually expect by "fitting" including error bars (and a covariance matrix for experts....).
   Errors are *1-sigma* errors as they are calculated from the covariance matrix and not directly depend
   on the errors of the *.eY*. Still the relative weight of values according to *.eY* is relevant.
 - method='BFGS' Broyden–Fletcher–Goldfarb–Shanno (BFGS) algorithm.  Not as fast as 'leastsq' but gives also error bars.
 - method='differential_evolution' is a global optimization method using iterative improving candidate solutions.
   In general it needs a large number of function calls but may find a global minimum.
 - method=‘Nelder-Mead’,'CG', ..... other optimization methods.
   These are slower converging than 'leastsq' and give no error bars.
   Some require a gradient function ore more.
   They are more for advanced users if someone really knows why using it in special cases.

### Warning:: **Save results !!!**

- The resulting parameters with errors are in *.lastfit*. **Save the fit result !!!**
 
- I regularly have to ask PhD students "What are the errors ?" and they repeat all their work again.
 
- Fit results without Errors are meaningless.


**If the fit finishes it tells if there was success or if it failed. For error messages the final parameters are no valid fit results!!! Please always check this.**

### A typical example of several datasets from a SAXS measurement (polymer in solution)


In [ ]:
import sys
!{sys.executable} -m pip install jscatter 
%matplotlib notebook

import jscatter as js
import numpy as np
js.usempl(True)   # force matplotlib, not needed in Linux

#### Read data

Inspect attributes.

In [ ]:
data=js.dL(js.examples.datapath+'/polymer.dat')
data.showattr()

#### Merge equal Temperatures each measured with two detector distances

In [ ]:
data.mergeAttribut('Temp',limit=0.01,isort='X')

#### define model
q will get the X values from your data as numpy ndarray.

In [ ]:
def gCpower(q,I0,Rg,A,beta,bgr):
    """Model Gaussian chain  + power law and background"""
    gc=js.ff.gaussianChain(q=q,Rg=Rg)
    # add power law and background
    gc.Y=I0*gc.Y+A*q**beta+bgr
    # add attributes for later documentation, these are additional content of lastfit (see below)
    gc.A=A
    gc.I0=I0
    gc.bgr=bgr
    gc.beta=beta
    gc.comment=['gaussianChain with power law and bgr','a second comment']
    return gc

#### Make errplot and set limits
For log scale klick on the plot and use k,l keys to switch.

In [ ]:
data.makeErrPlot()           # additional errorplot with intermediate output
data.setlimit(bgr=[0,1])     # upper and lower soft limit

#### Here we use individual parameter ([]) for all except a common beta ( no [] )
- Please try removing the [] and play with it :-)
- mapNames tells that q is *.X* (maps model names to data names )
- condition limits the range to fit (may also contain something like (a.Y>0))

In [ ]:
data.fit(model=gCpower,
         freepar={'I0':[0.1],'Rg':[3],'A':[1],'bgr':[0.01],'beta':-3},
         fixpar={},
         mapNames={'q':'X'},
         condition =lambda a:(a.X>0.05) & (a.X<4))

To fix a parameter move it to fixpar dict (bgr is automatically extended)

In [ ]:
data.fit(model=gCpower,
         freepar={'I0':[0.1],'Rg':[3],'A':[1]},
         fixpar={'bgr':[0.001, 0.0008, 0.0009],'beta':-4},
         mapNames={'q':'X'},
         condition =lambda a:(a.X>0.05) & (a.X<4))

Result parameter and error (example)

In [ ]:
data.lastfit.Rg
data.lastfit.Rg_err
# as result dataArray
result=js.dA(np.c_[data.Tempmean,data.lastfit.Rg,data.lastfit.Rg_err].T)
# plot it
p=js.mplot()
p.Plot(result,sy=[1,0.5,4],li=[2,2,1])
p.Xaxis(label='Temperature /  Celsius')
p.Yaxis(label='$R_{gyration}\; /\;  nm$')
# save the fit result including parameters, errors and covariance matrix
# and your model description
data.lastfit.save('polymer_fitDebye.dat')

### You may want to fit dataArray in a dataList individually.
Do it in a loop.

In [ ]:
# from the above
for dat in data:
   dat.fit(model=gCpower,
        freepar={'I0':0.1,'Rg':3,'A':1,},
        fixpar={'bgr':0.001,'beta':-3},
        mapNames={'q':'X'},
        condition =lambda a:(a.X>0.05) & (a.X<4))

# each dataArray has its own .lastfit, .errPlot and so on
data[0].showlastErrPlot()
# for saving with individual names
for dat in data:
   dat.lastfit.save('fitresult_Temp%2g.dat' %(dat.Tempmean))

# to get a list of the lastfit
fitresult=js.dL([dat.lastfit for dat in data])


#### To get a list of the lastfit

In [ ]:
fitresult=js.dL([dat.lastfit for dat in data])
fitresult